# Initialisation de Spark

In [8]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("Exemples avec les formats de stockage") \
    .getOrCreate()
sc = spark.sparkContext

# Chargement des données
1. Créer un dataframe pour les prénoms

In [9]:
prenoms = spark.read.load("../data/prenomsParDeptsEtAnnees.parquet")
prenoms.createOrReplaceTempView("prenoms")
prenoms.show()

+----+-----------------+--------+---+-----+
|sexe|           prenom|effectif|dep|annee|
+----+-----------------+--------+---+-----+
|   1|            TOMMY|       6| 76| 1999|
|   2|          CYNTHIA|       6| 76| 1999|
|   2|            ENOLA|      11| 76| 1999|
|   2|          MALVINA|       4| 76| 1999|
|   1|   _PRENOMS_RARES|      25| 67| 1959|
|   1|          FRANCIS|     112| 67| 1959|
|   1|           HUGUES|       4| 67| 1959|
|   2|         MARYLINE|       3| 64| 1985|
|   2|       PHILIPPINE|       3| 64| 1985|
|   1|CHARLES-ALEXANDRE|       4| 75| 1989|
|   1|        NATHANIEL|       6| 75| 2017|
|   1|         ROBINSON|       6| 75| 2017|
|   1|             TONY|      13| 92| 2007|
|   2|          ASSETOU|       3| 92| 2007|
|   2|            JANNA|       7| 92| 2007|
|   2|           LESLIE|       4| 92| 2007|
|   2|      ALEXANDRINE|      10| 59| 1977|
|   2|     MARIE-HÉLÈNE|      20| 59| 1977|
|   1|            ELYAN|       3| 75| 2011|
|   1|           JOHNNY|       3

1. Faire de même pour les départements

In [10]:
depts = spark.read.load("../data/depts.parquet")
depts.createOrReplaceTempView("depts")
depts.show()

+---+---+--------+----+--------------------+--------------------+--------------------+
|dep|reg|cheflieu|tncc|                 ncc|              nccenr|             libelle|
+---+---+--------+----+--------------------+--------------------+--------------------+
| 52| 44|   52121|   3|         HAUTE MARNE|         Haute-Marne|         Haute-Marne|
| 53| 52|   53130|   3|             MAYENNE|             Mayenne|             Mayenne|
| 54| 44|   54395|   0|  MEURTHE ET MOSELLE|  Meurthe-et-Moselle|  Meurthe-et-Moselle|
| 55| 44|   55029|   3|               MEUSE|               Meuse|               Meuse|
| 56| 53|   56260|   2|            MORBIHAN|            Morbihan|            Morbihan|
| 57| 44|   57463|   3|             MOSELLE|             Moselle|             Moselle|
| 58| 27|   58194|   3|              NIEVRE|              Nièvre|              Nièvre|
| 59| 32|   59350|   2|                NORD|                Nord|                Nord|
| 60| 32|   60057|   5|                OISE

# Interroger les données

* La documentation sur Spark SQL est disponible [ici](https://spark.apache.org/docs/latest/sql-programming-guide.html) (de même pour [DataFrame](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.html))
* Pour chaque question, vous donnerez la réponse avec le DSL et en SQL

1. Donner, pour chaque prénom, son nombre d'occurences

## DSL

In [21]:
prenoms.groupBy("prenom").count().show()

+---------------+-----+
|         prenom|count|
+---------------+-----+
|          MALIA|    1|
|          SILOÉ|    1|
|        ASSETOU|    1|
|        JUSTINE|    5|
| _PRENOMS_RARES|   24|
|       ROBINSON|    1|
|          ENOLA|    1|
|         ANGÈLE|    7|
|          ELYAS|    1|
|       CONSUELO|    1|
|            ALI|    4|
|     PHILIPPINE|    2|
|           LOUP|    2|
|PIERRE-EMMANUEL|    1|
|          SOLAL|    1|
|      GENEVIÈVE|    3|
|          ISSAM|    2|
|      FRANCETTE|    2|
|          CYRIL|    6|
|MARIE-FRANÇOISE|    1|
+---------------+-----+
only showing top 20 rows


## SQL

In [35]:
spark.sql('SELECT prenom, COUNT(*) FROM prenoms GROUP BY prenom').show()

+---------------+--------+
|         prenom|count(1)|
+---------------+--------+
|          MALIA|       1|
|          SILOÉ|       1|
|        ASSETOU|       1|
|        JUSTINE|       5|
| _PRENOMS_RARES|      24|
|       ROBINSON|       1|
|          ENOLA|       1|
|         ANGÈLE|       7|
|          ELYAS|       1|
|       CONSUELO|       1|
|            ALI|       4|
|     PHILIPPINE|       2|
|           LOUP|       2|
|PIERRE-EMMANUEL|       1|
|          SOLAL|       1|
|      GENEVIÈVE|       3|
|          ISSAM|       2|
|      FRANCETTE|       2|
|          CYRIL|       6|
|MARIE-FRANÇOISE|       1|
+---------------+--------+
only showing top 20 rows


1. Donner le nombre total de naissances avec un prénom féminin

## DSL

In [34]:
prenoms.filter(prenoms.sexe == 2).groupBy("sexe").count().show()

+----+-----+
|sexe|count|
+----+-----+
|   2| 2106|
+----+-----+



## SQL

In [33]:
spark.sql("SELECT COUNT(*) FROM prenoms WHERE sexe = 2").show()

+--------+
|count(1)|
+--------+
|    2106|
+--------+



1. Donner l'effectif maximal et minimal par prénom

## DSL

In [62]:
from pyspark.sql.functions import min, max

prenoms.groupBy("prenom").agg(min("effectif"), max("effectif")).show()

+---------------+-------------+-------------+
|         prenom|min(effectif)|max(effectif)|
+---------------+-------------+-------------+
|          MALIA|            5|            5|
|          SILOÉ|            3|            3|
|        ASSETOU|            3|            3|
|        JUSTINE|            3|           16|
| _PRENOMS_RARES|            3|          777|
|       ROBINSON|            6|            6|
|          ENOLA|           11|           11|
|         ANGÈLE|            3|           24|
|          ELYAS|            3|            3|
|       CONSUELO|            5|            5|
|            ALI|            3|           14|
|     PHILIPPINE|            3|            5|
|           LOUP|            3|            4|
|PIERRE-EMMANUEL|            4|            4|
|          SOLAL|           44|           44|
|      GENEVIÈVE|           47|           81|
|          ISSAM|            4|           16|
|      FRANCETTE|            3|            5|
|          CYRIL|            3|   

## SQL

In [64]:
spark.sql("SELECT prenom, MIN(effectif), MAX(effectif) FROM prenoms GROUP BY prenom").show()

+---------------+-------------+-------------+
|         prenom|min(effectif)|max(effectif)|
+---------------+-------------+-------------+
|          MALIA|            5|            5|
|          SILOÉ|            3|            3|
|        ASSETOU|            3|            3|
|        JUSTINE|            3|           16|
| _PRENOMS_RARES|            3|          777|
|       ROBINSON|            6|            6|
|          ENOLA|           11|           11|
|         ANGÈLE|            3|           24|
|          ELYAS|            3|            3|
|       CONSUELO|            5|            5|
|            ALI|            3|           14|
|     PHILIPPINE|            3|            5|
|           LOUP|            3|            4|
|PIERRE-EMMANUEL|            4|            4|
|          SOLAL|           44|           44|
|      GENEVIÈVE|           47|           81|
|          ISSAM|            4|           16|
|      FRANCETTE|            3|            5|
|          CYRIL|            3|   

1. Donner, pour chaque nom de département, le nombre moyen de prénoms depuis l'année 2000

## DSL

In [72]:
from pyspark.sql.functions import mean

prenoms.filter(prenoms.annee >= 2000).join(depts, prenoms.dep == depts.dep).groupBy("libelle").agg(mean("effectif")).show()

+--------------------+------------------+
|             libelle|     avg(effectif)|
+--------------------+------------------+
|    Bouches-du-Rhône|12.735294117647058|
|          Val-d'Oise|11.233333333333333|
|                Eure|21.928571428571427|
|                Nord|              24.8|
|              Guyane|               5.0|
|                Aube|              14.9|
|       Pas-de-Calais|15.136363636363637|
|               Somme| 9.833333333333334|
|        Val-de-Marne|           16.6875|
|               Paris| 16.23913043478261|
|            Yvelines| 7.791666666666667|
|             Moselle|             11.65|
|      Hauts-de-Seine|             13.75|
|               Loire|            7.1875|
|     Ille-et-Vilaine|             26.75|
|Territoire de Bel...|               4.6|
|       Haute-Garonne|15.090909090909092|
|               Rhône|           12.5625|
|    Loire-Atlantique| 16.09090909090909|
|                Oise| 5.571428571428571|
+--------------------+------------

## SQL

In [74]:
spark.sql("SELECT libelle, AVG(effectif) FROM prenoms, depts WHERE prenoms.dep = depts.dep AND annee >= 2000 GROUP BY depts.libelle").show()

+--------------------+------------------+
|             libelle|     avg(effectif)|
+--------------------+------------------+
|    Bouches-du-Rhône|12.735294117647058|
|          Val-d'Oise|11.233333333333333|
|                Eure|21.928571428571427|
|                Nord|              24.8|
|              Guyane|               5.0|
|                Aube|              14.9|
|       Pas-de-Calais|15.136363636363637|
|               Somme| 9.833333333333334|
|        Val-de-Marne|           16.6875|
|               Paris| 16.23913043478261|
|            Yvelines| 7.791666666666667|
|             Moselle|             11.65|
|      Hauts-de-Seine|             13.75|
|               Loire|            7.1875|
|     Ille-et-Vilaine|             26.75|
|Territoire de Bel...|               4.6|
|       Haute-Garonne|15.090909090909092|
|               Rhône|           12.5625|
|    Loire-Atlantique| 16.09090909090909|
|                Oise| 5.571428571428571|
+--------------------+------------